In [14]:
%load_ext autoreload
%autoreload 2

!pip install -e ../final-assignment/

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Obtaining file:///C:/Users/PiotrGrabysz/Documents/PythonScripts/NYPD/final-assignment
  Attempting uninstall: Warsaw-buses
    Found existing installation: Warsaw-buses 0.0.1
    Uninstalling Warsaw-buses-0.0.1:
      Successfully uninstalled Warsaw-buses-0.0.1
  Running setup.py develop for Warsaw-buses


In [25]:
import numpy 
numpy.__version__

'1.19.2'

## Collecting the buses data

Lets download some buses data. The single record of such data consists of line number, brigade number, vehicle number, latitude, longitude and timestamp.

To download it, please use the command `collect_buses`. There are several parameters to specify:

In [2]:
!collect_buses --help

usage: collect_buses [-h] [--dir_to_save DIR_TO_SAVE] [--time_step TIME_STEP]
                     [--how_long HOW_LONG]

optional arguments:
  -h, --help            show this help message and exit
  --dir_to_save DIR_TO_SAVE, -d DIR_TO_SAVE
                        A directory where the files with all the buses (or/and
                        trams) will be stored.
  --time_step TIME_STEP
                        The website will be requested every time_step (in
                        seconds) or longer, if processing and /saving the data
                        takes more time than time_step.
  --how_long HOW_LONG   For how long the request will be send to the website
                        (in seconds).


Lets collect these data for one minute:

In [6]:
!collect_buses -d ./data/buses_one_minute --how_long 60.0

Done! The buses data are saved in the directory: 'C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\buses_one_minute'


Every bus is saved in the separate file under the path: 'dir_to_save/line_number/brigade_number/vehicle_number.txt'.

In [11]:
sample_bus = pd.read_csv("./data/buses_one_minute/102/1/9411.txt", 
                        names=["Lat", "Lon", "Time"])
sample_bus.head()

,Lat,Lon,Time
0,52.237041,21.018202,2021-02-20 17:57:40
1,52.237156,21.019157,2021-02-20 17:57:49
2,52.237194,21.019484,2021-02-20 17:58:02
3,52.237194,21.019503,2021-02-20 17:58:14
4,52.237263,21.019985,2021-02-20 17:58:24


## Collecting the timetables

The api https://api.um.warszawa.pl provides three different type of data referring to timetables:

+ stops. It consists of each stop latitude and longitude, street_id, direction, name, stop_id, stop_nr

+ lines connected with the given stop. One should know stop_id and stop_nr to send this request

+ schedule for the given line at the given stop. One should know stop_id, stop_nr and line number to send this request. This data consist of brigade number, scheduled time of arrival and some other infos I don't use

My idea is to get the stops first (which I refer stops_coord to), iterate through every row of stops to get the numbers of lines going to this stop and then, get the schedule for every pair (stop, line). I save them in 'dir_to_timetables/line_number/stopId_stopNr.json'. Later on in the run_data_analysis I can easily create schedule of each brigade.

The stops_coord are needed later and they are saved in 'dir_to_timetables/stops_coord.json'.

But beware! Collecting all the timetables lasts for about two hours (there is nearly 8 000 stops, several lines are connected with each of them which results in tens of thousands requests).

In [15]:
!collect_timetables --help

usage: collect_timetables [-h] [--dir_to_save DIR_TO_SAVE] [--verbose]
                          [--repeat_rows_file REPEAT_ROWS_FILE]
                          [--resume_from_row RESUME_FROM_ROW]

optional arguments:
  -h, --help            show this help message and exit
  --dir_to_save DIR_TO_SAVE, -d DIR_TO_SAVE
                        A directory where all the timetables are stored in.
  --verbose, -v         Prints the name of each stop it is processing.
  --repeat_rows_file REPEAT_ROWS_FILE, -r REPEAT_ROWS_FILE
                        The file informing which rows from stops_coord should
                        be downloaded. For this version of the package please
                        use the 'dir_to_save/errors_log.txt' file generated by
                        collect_timetables. Defaults to None, which means that
                        every stop is processed.
  --resume_from_row RESUME_FROM_ROW, -r_row RESUME_FROM_ROW
                        If the downloading broke afte

If any request error happens during downloading, it's written into the 'dir_to_save/errors_log.txt' file with the information indicating at which row of stops_coord the error occured. You can use it later to download this missing data.